In [ ]:
# repo_dir = "Repos"   # Set this to be where your github repos are located.
# %load_ext autoreload
# %autoreload 2

# # Update the load path so python can find modules for the model
# import sys
# from pathlib import Path
# sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))
# sys.path.insert(0, str(Path.home() / repo_dir / "deriva-ml"))

In [ ]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI

import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva_ml import DatasetBag, Workflow, ExecutionConfiguration
from deriva_ml import MLVocab as vc
from deriva_ml.deriva_definitions import ColumnDefinition, BuiltinTypes
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [ ]:
# Login
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
host = 'dev.eye-ai.org'
# host = 'www.eye-ai.org'
catalog_id = "eye-ai"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

In [ ]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

# Configuration

In [ ]:
# RID of source dataset, if any.
source_dataset = '2-AGAW'

EA.add_term(vc.workflow_type, "Test Workflow", description="A test Workflow for new DM")
# Workflow instance
test_workflow = Workflow(
    name="Test New Workflow-multimodal",
    url="https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/templates/template_multimodal.ipynb",
    workflow_type="Test Workflow"
)
# Configuration instance.

# Set to False if you only need the metadata from the bag, and not the assets.
download_assets = True

config = ExecutionConfiguration(
    # Comment out the following line if you don't need the assets.
    datasets=[source_dataset] if download_assets else [],
    assets=['2-4JR6'],
    workflow=test_workflow,
    description="Template instance of a feature creation workflow")

# Initialize execution
execution = EA.create_execution(config)


In [ ]:
print(execution)

# Create DatasetBag

In [ ]:
ds_bag = DatasetBag(configuration_record.bag_paths[0])

# Methods for multimodal data

### Extract modality

In [ ]:
modalities = EA.extract_modality(ds_bag)

In [ ]:
modalities

### Retrive multimodal wide table

In [ ]:
wide = EA.multimodal_wide(ds_bag)

In [ ]:
wide.columns

In [ ]:
import tensorflow as tf


In [ ]:
# wide['Condition_Label'].values  
# wide[['Condition_Label']].values 
# wide.drop(columns=['Condition_Label']).values
# columns = ['RID_Subject', 'Image_Side', 'Subject_Gender', 'MD', 'Average_C/D_Ratio']
from sklearn.preprocessing import LabelEncoder


In [ ]:
columns = ['MD', 'Average_C/D_Ratio']
wide['Condition_Label'] = LabelEncoder().fit_transform(wide['Condition_Label'])
tfds = tf.data.Dataset.from_tensor_slices(
    {
        'id': wide['RID_Subject'].values,
        'Image_Side': wide['Image_Side'].values,
        'Subject_Gender': wide['Subject_Gender'].values,
        'MD': wide['MD'].values,
        'Average_C/D_Ratio': wide['Average_C/D_Ratio'].values,
        'label': wide['Condition_Label'].values
    }
)

### Severity analysis

In [ ]:
EA.severity_analysis(ds_bag)

# Upload results

In [ ]:
# crete asset path
asset_path = execution.asset_directory('Multimodal_Analysis')
wide.to_csv(asset_path/'wide_test.csv')

# upload assets to catalog
execution.upload_execution_outputs(clean_folder=True)